In [1]:
%run common.ipynb

[tokenizer.decode(i) for i in tokenizer.get_batch_data(prefix=True)[1]][:10]

['S大写:1882=柒伍贰捌柒伍贰捌柒伍贰捌柒伍贰捌EP',
 'S字母:6947=wuuopuuopuuopuuiiE',
 'S数字:1913=7652765276527652EP',
 'S字母:3075=qwepqwepqwepqweppE',
 'S小写:3408=一三六三三三六三三三六三三三六三二E',
 'S数字:6841=27366736673667364E',
 'S数字:2796=11185118511851184E',
 'S大写:5667=贰贰陆柒零贰陆柒零贰陆柒零贰陆陆捌E',
 'S字母:8800=etwpetwpetwpetwppE',
 'S大写:6691=贰陆柒陆陆陆柒陆陆陆柒陆陆陆柒陆肆E']

In [2]:
model_ppo = ModelPPO(torch.load('gen.model'))
model_ppo_ref = ModelPPO(torch.load('gen.model'))

for i in model_ppo_ref.parameters():
    i.requires_grad_(False)

/root/anaconda3/envs/pt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_kl(a, b):
    method = 'kl'

    if method == 'kl':
        return a - b

    if method == 'abs':
        return (a - b).abs()

    if method == 'mse':
        return (a - b).square() * 0.5

    if method == 'full':
        return torch.nn.functional.kl_div(a,
                                          b,
                                          log_target=True,
                                          reduction='none')


get_kl(torch.randn(15), torch.zeros(15))

tensor([-0.6798, -0.9093, -0.2102,  0.4281, -0.0594,  1.7366, -0.4399,  0.5919,
         1.0430, -1.0704, -0.7517, -1.2599,  1.4613, -1.1815,  0.5282])

In [4]:
from trl.core import clip_by_value, logprobs_from_logits, masked_mean, masked_whiten


class PPOTrainer:

    def __init__(self):
        self.optimizer = torch.optim.Adam(model_ppo.parameters(), lr=1e-5)

    def step(self, question, answer, reward):
        with torch.no_grad():
            #编码
            token = [q.tolist() + a.tolist() for q, a in zip(question, answer)]
            input_ids, attention_mask = tokenizer.batch_pad(token=token)
            del token
            input_ids = torch.LongTensor(input_ids).to(device)
            attention_mask = torch.LongTensor(attention_mask).to(device)

            #question和answer不需要内容,只需要长度信息即可
            lens_q = [len(i) for i in question]
            lens_a = [len(i) for i in answer]
            del question
            del answer

            #根据question计算answer的概率,并计算每个动作的分数
            prob_log, value, mask = self.batched_forward_pass(
                model_ppo, input_ids, attention_mask, lens_q, lens_a)

            #使用ref模型计算概率,这是为了计算kl散度
            prob_log_ref, _, _ = self.batched_forward_pass(
                model_ppo_ref, input_ids, attention_mask, lens_q, lens_a)

            #计算两份概率的kl散度,并融入reward
            reward = self.compute_rewards(reward, prob_log, prob_log_ref, mask)

            #计算delta和target,用于计算loss
            value, delta, target = self.compute_advantages(value, reward, mask)

        #每批数据循环N次模型
        for _ in range(4):
            #每次算一个数据
            for i in range(len(input_ids)):
                #重新计算概率和value
                prob_log_new, value_new, _ = self.batched_forward_pass(
                    model_ppo, input_ids[i].unsqueeze(0),
                    attention_mask[i].unsqueeze(0), [lens_q[i]], [lens_a[i]])

                #根据新旧概率求出变化率,进而求出loss
                #根据target和value的差可以计算出另外一份loss
                loss = self.get_loss(prob_log[i].unsqueeze(0),
                                     value[i].unsqueeze(0), prob_log_new,
                                     value_new, mask[i].unsqueeze(0),
                                     delta[i].unsqueeze(0),
                                     target[i].unsqueeze(0))

                if not loss:
                    continue

                loss.backward()
                #torch.nn.utils.clip_grad_norm_(model_ppo.parameters(), 1.0)
                self.optimizer.step()
                self.optimizer.zero_grad()

    def batched_forward_pass(self, model, input_ids, attention_mask, lens_q,
                             lens_a):
        logits, value = model(input_ids=input_ids,
                              attention_mask=attention_mask)

        #取每个字的概率对数
        prob_log = logprobs_from_logits(logits[:, :-1], input_ids[:, 1:])

        #是预测结果并且不是PAD的位置是1
        mask = torch.zeros_like(attention_mask)
        mask[:, :-1] = attention_mask[:, 1:]
        for i in range(len(input_ids)):
            start = lens_q[i] - 1
            end = start + lens_a[i]
            mask[i, :start] = 0
            mask[i, end:] = 0

        #对最后一个字的预测没有意义,直接丢弃
        value = value[:, :-1]
        mask = mask[:, :-1]

        return prob_log, value, mask

    def compute_rewards(self, reward, prob_log, prob_log_ref, mask):
        reward_kl = []

        for i in range(len(reward)):
            #求两份概率的kl散度
            kl = get_kl(prob_log[i], prob_log_ref[i]) * -0.2

            #把reward加在最后一个字的kl散度上
            if (mask[i] == 0).all():
                #print('all 0')
                idx = 0
            else:
                idx = mask[i].nonzero()[-1].item()
            kl[idx] += reward[i]

            reward_kl.append(kl)

        return torch.stack(reward_kl)

    def compute_advantages(self, value, reward_kl, mask):
        value = value * mask
        reward_kl = reward_kl * mask

        delta = []
        lens = reward_kl.shape[1]

        #从后往前遍历
        for i in reversed(range(lens)):
            #取下一时刻的value,如果已经是最后一个时刻,则value_next是0
            #因为整个循环是从后往前,所以第0次是0,其他时刻取value
            value_next = 0
            if i < lens - 1:
                value_next = value[:, i + 1]

            #value = gamma*下一时刻的value + reward
            #理论上相等,这里的差定义为delta,这里gamma是1,所以省略了
            d = reward_kl[:, i] + value_next - value[:, i]

            #取最后一个delta,如果还没有,则初始化为0
            last_d = 0
            if delta:
                last_d = delta[-1]

            #delta是从后往前传递的,这里的系数衡量了前后动作的因果关联性
            delta.append(d + 0.95 * last_d)

        #翻转顺序
        delta = torch.stack(delta[::-1]).transpose(0, 1)

        #定义target,它估计了理想的value值
        target = delta + value
        delta = masked_whiten(delta, mask)

        return value, delta, target

    def get_loss(self, prob_log, value, prob_log_new, value_new, mask, delta,
                 target):

        #对数概率,相除变相减,取exp后还原为商,即两个模型输出logits的变化率
        ratio = (prob_log_new - prob_log).exp()

        #如果变化率太过于剧烈,可能是发生了震荡,跳过
        if masked_mean(ratio, mask).item() > 10:
            #print('skip', masked_mean(ratio, mask).item())
            return None

        #先算两个value的loss,简单的算mse loss就可以了
        loss_vf1 = (value_new - target)**2
        #数值裁剪,很显然是为了缓解自举
        loss_vf2 = clip_by_value(value_new, value - 0.2, value + 0.2)
        loss_vf2 = (loss_vf2 - target)**2
        #两份loss取大的,还是为了缓解自举
        loss_vf = 0.5 * masked_mean(torch.max(loss_vf1, loss_vf2), mask)

        #计算ppo loss
        loss_surr1 = -delta * ratio
        #数值裁剪,很显然是为了缓解自举
        loss_surr2 = -delta * ratio.clamp(0.8, 1.2)
        loss_surr = masked_mean(torch.max(loss_surr1, loss_surr2), mask)

        return loss_surr + 0.1 * loss_vf


trainer = PPOTrainer()

trainer

In [5]:
model_cls = torch.load('cls.model')
model_cls.to(device)

for i in model_cls.parameters():
    i.requires_grad_(False)

In [6]:
def get_question():
    label, question, _ = tokenizer.get_batch_data(prefix=True)
    label = torch.LongTensor(label).to(device)

    #只要问题部分,等号后面的内容切除
    question = [i[:i.index(tokenizer.encoder['=']) + 1] for i in question]
    question = [torch.LongTensor(i).to(device) for i in question]

    return label, question


label, question = get_question()

label, question[:10]

(tensor([0, 1, 2, 2, 2, 3, 3, 0, 3, 0, 1, 0, 1, 2, 2, 1, 3, 3, 1, 3, 3, 3, 0, 1,
         0, 1, 1, 0, 3, 0, 1, 2, 3, 0, 1, 2, 0, 1, 0, 2, 3, 2, 2, 1, 3, 0, 3, 3,
         3, 2, 1, 3, 2, 3, 1, 1, 0, 3, 2, 3, 0, 0, 1, 0], device='cuda:0'),
 [tensor([ 1, 44, 45, 50,  6, 10, 11,  8, 51], device='cuda:0'),
  tensor([ 1, 45, 49, 50,  9,  6,  4, 11, 51], device='cuda:0'),
  tensor([ 1, 48, 47, 50,  9,  7,  8,  8, 51], device='cuda:0'),
  tensor([ 1, 48, 47, 50,  9,  4,  8,  6, 51], device='cuda:0'),
  tensor([ 1, 48, 47, 50, 11,  4,  7, 10, 51], device='cuda:0'),
  tensor([ 1, 46, 47, 50,  6, 11,  8,  6, 51], device='cuda:0'),
  tensor([ 1, 46, 47, 50, 13,  7, 11,  8, 51], device='cuda:0'),
  tensor([ 1, 44, 45, 50,  6,  5,  8, 10, 51], device='cuda:0'),
  tensor([ 1, 46, 47, 50,  6,  4,  6,  4, 51], device='cuda:0'),
  tensor([ 1, 44, 45, 50, 12,  8, 12,  8, 51], device='cuda:0')])

In [7]:
#如果question的长度确定,这里可以转换成批运算
def get_answer(question):
    answer = [generate(model_ppo.model_gen, i.unsqueeze(0)) for i in question]

    #裁剪,只要生成的部分
    answer = [a[0, len(q):] for q, a in zip(question, answer)]

    return answer


answer = get_answer(question)

answer[:10]

[tensor([ 5,  4, 11,  5, 11,  4, 11,  5, 11,  4, 11,  5, 11,  4, 11,  5, 10,  2],
        device='cuda:0'),
 tensor([ 6,  4, 12,  7,  4,  4, 12,  7,  4,  4, 12,  7,  4,  4, 12,  6, 12,  2],
        device='cuda:0'),
 tensor([26, 25, 27, 31, 32, 25, 27, 31, 32, 25, 27, 31, 32, 25, 27, 31, 30,  2],
        device='cuda:0'),
 tensor([26, 24, 25, 31, 24, 24, 25, 31, 24, 24, 25, 31, 24, 24, 25, 30, 32,  2],
        device='cuda:0'),
 tensor([16, 22, 15, 18, 20, 22, 15, 18, 20, 22, 15, 18, 20, 22, 15, 18, 18,  2],
        device='cuda:0'),
 tensor([35, 34, 43, 40, 43, 34, 43, 40, 43, 34, 43, 40, 43, 34, 43, 40, 42,  2],
        device='cuda:0'),
 tensor([27, 31, 28, 33, 33, 31, 28, 33, 33, 31, 28, 33, 33, 31, 28, 33, 30,  2],
        device='cuda:0'),
 tensor([32, 29, 31, 28, 32, 29, 31, 28, 32, 29, 31, 28, 32, 29, 31, 28,  2],
        device='cuda:0'),
 tensor([12,  4, 12,  4, 12,  4, 12,  4, 12,  4, 12,  4, 12,  4, 12,  4,  2],
        device='cuda:0'),
 tensor([ 7,  7, 13,  7, 13,  7, 13,

In [8]:
def get_reward(question, answer, label):
    token = [q.tolist() + a.tolist() for q, a in zip(question, answer)]

    input_ids, attention_mask = tokenizer.batch_pad(token=token)
    input_ids = torch.LongTensor(input_ids).to(device)
    attention_mask = torch.LongTensor(attention_mask).to(device)

    with torch.no_grad():
        logits = model_cls(input_ids=input_ids, attention_mask=attention_mask)

    return logits.gather(1, label.reshape(-1, 1)).squeeze(1)


reward = get_reward(question, answer, label)

reward

tensor([ 3.6872, -1.1460,  3.8057,  3.9364, -1.4702,  3.7345, -0.4405, -0.6043,
        -0.8919,  3.5087, -0.9511, -1.1840, -1.4771, -1.8334,  4.2692,  3.8740,
        -0.8011, -0.5006, -1.1768,  3.8684, -0.8430, -1.2840, -1.3416, -1.3015,
         3.8101, -1.0993, -1.4701,  3.8751, -0.8155, -0.6414, -0.9203, -0.4777,
         3.4747, -1.2405,  3.7453,  4.3441, -1.3180, -0.9811, -1.2982, -0.5892,
        -0.9671,  3.6851, -0.4399, -1.7880, -0.9857, -0.9828, -0.7380, -1.0874,
        -1.2675, -1.4632,  3.6310,  3.3023, -1.1148,  3.3922, -1.2878,  3.8767,
        -1.4827, -0.4335, -1.2032,  3.4332,  3.5496, -1.2879, -1.3275, -1.1620],
       device='cuda:0')

In [9]:
for epoch in range(2000):
    label, question = get_question()
    answer = get_answer(question)
    reward = get_reward(question, answer, label)

    trainer.step(question, answer, reward)

    if epoch % 100 == 0:
        print(epoch, reward.mean().item())
        for _, q, a, r in zip(range(2), question, answer, reward):
            q = tokenizer.decode(q.tolist())
            a = tokenizer.decode(a.tolist())
            r = r.item()
            print(q, a, r)

model_ppo.to('cpu')
torch.save(model_ppo, 'ppo.model')

0 0.2149033397436142
S字母:8964= etitotitotitotityE 3.619676351547241
S小写:3435= qeurqeurqeurqeurpE -1.2818214893341064
100 0.5845248699188232
S字母:2815= 一一二六一一二六一一二六一一二六〇E -1.7823433876037598
S小写:4382= 一七五二九七五二九七五二九七五二八E 4.079746246337891
200 0.43839702010154724
S小写:8092= 32371237123712368E -0.2243329882621765
S大写:1042= 4068406840684068E -1.1130495071411133
300 1.0763766765594482
S数字:4186= 17745774577457744E 3.7182233333587646
S字母:3719= 一四八七七四八七七四八七七四八七六E -1.4818665981292725
400 1.4338525533676147
S小写:3025= 一二一〇一二一〇一二一〇一二一〇〇E 4.044591903686523
S字母:7551= 30207020702070204E -1.1103068590164185
500 2.2621121406555176
S字母:9061= 36247624762476244E -1.077667236328125
S大写:1859= ureyureyureyureyE -1.058005928993225
600 3.02119517326355
S字母:7934= equeoqueoqueoqueyE 4.11110782623291
S字母:1110= rrrprrrprrrprrrpE 3.9544923305511475
700 3.7655293941497803
S大写:5480= 贰壹玖贰贰壹玖贰贰壹玖贰贰壹玖贰零E 3.579922914505005
S字母:5507= wwpepwpepwpepwpwiE 3.8950295448303223
800 3.623467445373535
S小写:4569= 一八二七七八二七七八二七七八二七六E 4.2